# Hospital Length of Stay

In order for hospitals to optimize resource allocation, it is important to predict accurately how long a newly admitted patient will stay in the hospital.

This notebook takes advantage of the power of SQL Server and RevoScaleR (Microsoft R Server). The tables are all stored in a SQL Server, and most of the computations are done by loading chunks of data in-memory instead of the whole dataset.

It does the following: 

 * **Step 0: Packages and Compute Contexts**
 * **Step 1: Processing and Cleaning**
 * **Step 2: Feature Engineering**
 * **Step 3: Training and Evalutating a Random Forest (rxDForest) and Boosted Trees (rxFastTrees)**

## Step 0: Packages and Compute Contexts

#### In this step, we set up the connection string to access a SQL Server Database and load the necessary packages. 

In [ ]:
# WARNING.
# We recommend not using Internet Explorer as it does not support plotting, and may crash your session.

In [ ]:
# Load packages.
library(RevoScaleR)
library("MicrosoftML")

In [ ]:
# Choose a database name and create it. 
db <- "Hospital_R"

## Connect to the master database only to create a new database. Change UID and PWD if you modified them. 
connection_string <- "Driver=SQL Server;Server=localhost;Database=master;Trusted_Connection=True"

## Open a connection with SQL Server to be able to write queries with the rxExecuteSQLDDL function.
outOdbcDS <- RxOdbcData(table = "NewData", connectionString = connection_string, useFastRead=TRUE)
rxOpen(outOdbcDS, "w")

query <- sprintf( "if not exists(SELECT * FROM sys.databases WHERE name = '%s') CREATE DATABASE %s;", db, db)

## Create database. 
rxExecuteSQLDDL(outOdbcDS, sSQLString = query)

In [ ]:
# Define Compute Contexts: user to input Server Name, database name, UID and Password. 
connection_string <- sprintf("Driver=SQL Server;Server=localhost;Database=%s;UID=bob;PWD=ChangePassw)rd12", db)
sql <- RxInSqlServer(connectionString = connection_string)
local <- RxLocalSeq()

# Open a connection with SQL Server to be able to write queries with the rxExecuteSQLDDL function in the new database.
outOdbcDS <- RxOdbcData(table = "NewData", connectionString = connection_string, useFastRead=TRUE)
rxOpen(outOdbcDS, "w")

#### The function below can be used to get the top n rows of a table stored on SQL Server. 
#### You can execute this cell throughout your progress by removing the comment "#", and inputting:
#### - the table name.
#### - the number of rows you want to display.

In [ ]:
 display_head <- function(table_name, n_rows){
   table_sql <- RxSqlServerData(sqlQuery = sprintf("SELECT TOP(%s) * FROM %s", n_rows, table_name), connectionString = connection_string)
   table <- rxImport(table_sql)
   print(table)
}

# table_name <- "insert_table_name"
# n_rows <- 10
# display_head(table_name, n_rows)

## Step 1: Pre-Processing and Cleaning

In this step, we: 

**1.** Upload the data set to SQL.

**2.** Clean the merged data set: we replace NAs with the mode (categorical variables) or mean (continuous variables).

**Input:**  Data Set LengthOfStay.csv

**Output:** Cleaned raw data set LoS.

In [ ]:
# Set the compute context to Local. 
rxSetComputeContext(local)

In [ ]:
# Upload the data set to SQL.

## Specify the desired column types. 
## When uploading to SQL, Character and Factor are converted to nvarchar(255), Integer to Integer and Numeric to Float. 
column_types <-  c(eid = "integer",               
                   vdate = "character",           
                   rcount = "character",        
                   gender = "factor",            
                   dialysisrenalendstage = "factor",             
                   asthma = "factor",                
                   irondef = "factor",                   
                   pneum = "factor",                 
                   substancedependence = "factor",                  
                   psychologicaldisordermajor = "factor",             
                   depress = "factor",           
                   psychother = "factor",        
                   fibrosisandother = "factor",          
                   malnutrition = "factor",                               
                   hemo = "factor",            
                   hematocrit = "numeric",           
                   neutrophils = "numeric",           
                   sodium = "numeric",          
                   glucose = "numeric",             
                   bloodureanitro = "numeric",                 
                   creatinine = "numeric",                 
                   bmi = "numeric",                 
                   pulse = "numeric",                  
                   respiration = "numeric",                  
                   secondarydiagnosisnonicd9 = "factor",
                   discharged = "character",
                   facid = "factor",
                   lengthofstay = "integer")


## Point to the input data set while specifying the classes.
#file_path = "..\\\\Data\"\n
#path = 'c:\Solutions\Hospital\Data'
LoS_text <- RxTextData(file = 'c:\\Solutions\\Hospital\\Data\\LengthOfStay.csv', colClasses = column_types)

## Upload the table to SQL. 
LengthOfStay_sql <- RxSqlServerData(table = "LengthOfStay", connectionString = connection_string)
rxDataStep(inData = LoS_text, outFile = LengthOfStay_sql, overwrite = TRUE)

print("Data exported to SQL")

In [ ]:
# Determine if LengthOfStay has missing values

table <- "LengthOfStay"

# First, get the names and types of the variables to be treated.
data_sql <- RxSqlServerData(table = table, connectionString = connection_string)
col <- rxCreateColInfo(data_sql)

# Then, get the names of the variables that actually have missing values. Assumption: no NA in eid, lengthofstay, or dates. 
colnames <- names(col)
var <- colnames[!colnames %in% c("eid", "lengthofstay", "vdate", "discharged")]
formula <- as.formula(paste("~", paste(var, collapse = "+")))
summary <- rxSummary(formula, data_sql, byTerm = TRUE)
var_with_NA <- summary$sDataFrame[summary$sDataFrame$MissingObs > 0, 1] 

if(length(var_with_NA) == 0){
  print("No missing values.")
  missing <- 0
  
} else{
  print("Variables containing missing values are:")
  print(var_with_NA)
  print("The NAs will be replaced with the mode or mean.")
  missing <- 1
}    

In [ ]:
# If applicable, NULL is replaced with the mode (categorical variables: integer or character) or mean (continuous variables).

if(missing == 0){
    print("Nothing to clean")
    LengthOfStay_cleaned_sql <- RxSqlServerData(table = table, connectionString = connection_string)
} else{
# Get the variables types (categortical vs. continuous) 
categ_names <- c()
contin_names <- c()
  for(name in var_with_NA){
    if(col[[name]]$type == "numeric"){
      contin_names[length(contin_names) + 1] <- name
    } else{
      categ_names[length(categ_names) + 1] <- name
    }
  }
# For Categoricals: Compute the mode of the variables with SQL queries in table Modes. We then import Modes. 
rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("DROP TABLE if exists Modes;"
                                              , sep=""))

rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("CREATE TABLE Modes
                                              (name varchar(30),
                                              mode varchar(30));"
                                              , sep=""))

for(name in categ_names){
  rxExecuteSQLDDL(outOdbcDS, sSQLString = sprintf("INSERT INTO Modes
                                                  SELECT '%s', mode
                                                  FROM (SELECT TOP(1) %s as mode, count(*) as cnt
                                                  FROM %s
                                                  GROUP BY %s 
                                                  ORDER BY cnt desc) as t;",name, name, table, name))
}
Modes_sql <- RxSqlServerData(table = "Modes", connectionString = connection_string) 
Modes <- rxImport(Modes_sql)

# For Continuous: Compute the mode of the variables with SQL queries in table Means. We then import Means. 
rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("DROP TABLE if exists Means;"
                                              , sep=""))

rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("CREATE TABLE Means
                                              (name varchar(30),
                                              mean float);"
                                              , sep=""))

for(name in contin_names){
  rxExecuteSQLDDL(outOdbcDS, sSQLString = sprintf("INSERT INTO Means
                                                  SELECT '%s', mean
                                                  FROM (SELECT AVG(%s) as mean
                                                  FROM %s) as t;",name, name, table))
}
Means_sql <- RxSqlServerData(table = "Means", connectionString = connection_string) 
Means <- rxImport(Means_sql)
 
# Function to replace missing values with the mode (categorical variables) or mean (continuous variables)
fill_NA_mode_mean <- function(data){
  data <- data.frame(data)
  for(j in 1:length(categ)){
    row_na <- which(is.na(data[,categ[j]]) == TRUE) 
    if(length(row_na > 0)){
      data[row_na, categ[j]] <- subset(Mode, name == categ[j])[1,2]
    }
  }
  for(j in 1:length(contin)){
    row_na <- which(is.na(data[,contin[j]]) == TRUE) 
    if(length(row_na > 0)){
      data[row_na, contin[j]] <- subset(Mean, name == contin[j])[1,2]
    }
  }
  return(data)
}

# Apply this function to LeangthOfStay by wrapping it up in rxDataStep. Output is written to LoS0.   
LoS0_sql <- RxSqlServerData(table = "LoS0", connectionString = connection_string)
rxDataStep(inData = LengthOfStay_sql , outFile = LoS0_sql, overwrite = TRUE, transformFunc = fill_NA_mode_mean, 
           transformObjects = list(categ = categ_names, contin = contin_names, Mode = Modes, Mean = Means))
   
LengthOfStay_cleaned_sql <- RxSqlServerData(table = "LoS0", connectionString = connection_string)    
    
print("Data cleaned")
}


## Step 2: Feature Engineering

In this step, we:

**1.** Standardize the continuous variables (Z-score).

**2.** Create the variable number_of_issues: the number of preidentified medical conditions.

**Input:** Data set before feature engineering LengthOfStay.

**Output:** Data set with new features LoS.

In [ ]:
# Get the mean and standard deviation of those variables.
names <- c("hematocrit", "neutrophils", "sodium", "glucose", "bloodureanitro",
           "creatinine", "bmi", "pulse", "respiration")
summary <- rxSummary(formula = ~., LengthOfStay_cleaned_sql, byTerm = TRUE)$sDataFrame
Statistics <- summary[summary$Name %in% names,c("Name", "Mean", "StdDev")]

# Function to standardize
standardize <- function(data){
  data <- data.frame(data)
  for(n in 1:nrow(Stats)){
    data[[Stats[n,1]]] <- (data[[Stats[n,1]]] - Stats[n,2])/Stats[n,3]
    }
  return(data)
}

### If LoS Table or view as already been created in SQL , get rid of it 
rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("IF (SELECT count(*) FROM sys.objects where type = 'U' and name = 'LoS')  = 1 
                                                BEGIN DROP TABLE LoS END;"
                                              , sep=""))
rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("IF (SELECT count(*) FROM sys.views where name = 'LoS')  = 1 
                                                BEGIN DROP VIEW LoS END;"
                                              , sep=""))


# Apply this function to the cleaned table by wrapping it up in rxDataStep. Output is written to LoS.  
# At the same time, we create number_of_issues as the number of preidentified medical conditions.
# We also create lengthofstay_bucket as the bucketed version of lengthofstay for classification. 
LoS_sql <- RxSqlServerData(table = "LoS", connectionString = connection_string)
rxDataStep(inData = LengthOfStay_cleaned_sql , outFile = LoS_sql, overwrite = TRUE, transformFunc = standardize, 
           transformObjects = list(Stats = Statistics), transforms = list(
             number_of_issues = as.numeric(hemo) + as.numeric(dialysisrenalendstage) + as.numeric(asthma) + as.numeric(irondef) + 
                                as.numeric(pneum) + as.numeric(substancedependence) +
                                as.numeric(psychologicaldisordermajor) + as.numeric(depress) + as.numeric(psychother) + 
                                as.numeric(fibrosisandother) + as.numeric(malnutrition) 
             
           ))

           
# Converting number_of_issues to character with a SQL query because as.character in rxDataStep is crashing.           
rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("ALTER TABLE LoS ALTER COLUMN number_of_issues varchar(2);", sep=""))

print("Feature Engineering Completed")

## Step 3: Training and Evaluating the Models

In this step we:

**1.** Split randomly the data set LoS into a training (LoS_Train) and a testing (LoS_Test) set.
 
**2.** Train a Random Forest (rxDForest) and Boosted Trees (rxFastTrees) models on LoS_Train, and save them to SQL. 

**3.** Score the models on LoS_Test.

**Input:** Data set LoS.

**Output:** Random forest and Boosted Trees models saved to SQL and performance metrics.  

In [ ]:
# Point to the SQL table with the data set for modeling. Strings will be converted to factors.
LoS <- RxSqlServerData(table = "LoS", connectionString = connection_string, stringsAsFactors = T)

# Get variable names, types, and levels for factors.
column_info <- rxCreateColInfo(LoS)

print("Column information received")

In [ ]:
# Randomly split the data into a training set and a testing set, with a splitting % p.
# p % goes to the training set, and the rest goes to the testing set. Default is 70%. 

p <- "70" 

## Open a connection with SQL Server to be able to write queries with the rxExecuteSQLDDL function.
outOdbcDS <- RxOdbcData(table = "NewData", connectionString = connection_string, useFastRead=TRUE)
rxOpen(outOdbcDS, "w")

## Create the Train_Id table containing Lead_Id of training set. 
rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("DROP TABLE if exists Train_Id;", sep=""))

rxExecuteSQLDDL(outOdbcDS, sSQLString = sprintf(
  "SELECT eid
   INTO Train_Id
   FROM LoS
   WHERE ABS(CAST(BINARY_CHECKSUM(eid, NEWID()) as int)) %s < %s ;"
  ,"% 100", p ))

## Point to the training set. It will be created on the fly when training models. 
LoS_Train <- RxSqlServerData(  
  sqlQuery = "SELECT *   
              FROM LoS 
              WHERE eid IN (SELECT eid from Train_Id)",
  connectionString = connection_string, colInfo = column_info)

## Point to the testing set. It will be created on the fly when testing models. 
LoS_Test <- RxSqlServerData(  
  sqlQuery = "SELECT *   
              FROM LoS 
              WHERE eid NOT IN (SELECT eid from Train_Id)",
  connectionString = connection_string, colInfo = column_info)

print("Splitting completed")

In [ ]:
# Write the formula after removing variables not used in the modeling.
variables_all <- rxGetVarNames(LoS)
variables_to_remove <- c("eid", "vdate", "discharged", "facid")
traning_variables <- variables_all[!(variables_all %in% c("lengthofstay", variables_to_remove))]
formula <- as.formula(paste("lengthofstay ~", paste(traning_variables, collapse = "+")))

print("Formula written")

In [ ]:
# Compute Context is set to SQL for model training.
rxSetComputeContext(sql)

In [ ]:
# Train the Random Forest.
forest_model <- rxDForest(formula = formula,
                          data = LoS_Train,
                          nTree = 40,
                          minSplit = 10,
                          minBucket = 5,
                          cp = 0.00005,
                          seed = 5)

print("Training Regression RF done")

In [ ]:
# Save the Random Forest in SQL. The compute context is set to local in order to export the model. 
rxSetComputeContext(local)
saveRDS(forest_model, file = "forest_model.rds")
forest_model_raw <- readBin("forest_model.rds", "raw", n = file.size("forest_model.rds"))
forest_model_char <- as.character(forest_model_raw)
forest_model_sql <- RxSqlServerData(table = "Forest_ModelR", connectionString = connection_string) 
rxDataStep(inData = data.frame(x = forest_model_char ), outFile = forest_model_sql, overwrite = TRUE)

# Set back the compute context to SQL.
rxSetComputeContext(sql)

print("RF model uploaded to SQL")

In [ ]:
# Train the Boosted Trees model.
library("MicrosoftML")
boosted_model <- rxFastTrees(formula = formula,
                             data = LoS_Train,
                             type = c("regression"),
                             numTrees = 40,
                             learningRate = 0.2,
                             splitFraction = 5/24,
                             featureFraction = 1,
                             minSplit = 10)

print("Training Regression Boosted Trees done")

In [ ]:
# Save the Boosted Trees in SQL. The compute context is set to Local in order to export the model. 
rxSetComputeContext(local)
saveRDS(boosted_model, file = "boosted_model.rds")
boosted_model_raw <- readBin("boosted_model.rds", "raw", n = file.size("boosted_model.rds"))
boosted_model_char <- as.character(boosted_model_raw)
boosted_model_sql <- RxSqlServerData(table = "Boosted_ModelR", connectionString = connection_string) 
rxDataStep(inData = data.frame(x = boosted_model_char ), outFile = boosted_model_sql, overwrite = TRUE)

print("Boosted Trees model uploaded to SQL")

In [ ]:
# Write a function that computes regression performance metrics. 
evaluate_model <- function(observed, predicted, model) {
  mean_observed <- mean(observed)
  se <- (observed - predicted)^2
  ae <- abs(observed - predicted)
  sem <- (observed - mean_observed)^2
  aem <- abs(observed - mean_observed)
  mae <- mean(ae)
  rmse <- sqrt(mean(se))
  rae <- sum(ae) / sum(aem)
  rse <- sum(se) / sum(sem)
  rsq <- 1 - rse
  metrics <- c("Mean Absolute Error" = mae,
               "Root Mean Squared Error" = rmse,
               "Relative Absolute Error" = rae,
               "Relative Squared Error" = rse,
               "Coefficient of Determination" = rsq)
  print(model)
  print(metrics)
  print("Summary statistics of the absolute error")
  print(summary(abs(observed-predicted)))
  return(metrics)
}

In [ ]:
# Random Forest Scoring 

## Make Predictions, then import them into R. 
forest_prediction_sql <- RxSqlServerData(table = "Forest_Prediction", stringsAsFactors = T,
                                         connectionString = connection_string)

rxPredict(modelObject = forest_model,
          data = LoS_Test, 
          outData = forest_prediction_sql,
          overwrite = T, 
          type = "response",
          extraVarsToWrite = c("lengthofstay", "eid"))

## Compute the performance metrics of the model.
forest_prediction <- rxImport(inData = forest_prediction_sql)

forest_metrics <- evaluate_model(observed = forest_prediction$lengthofstay,
                                 predicted = forest_prediction$lengthofstay_Pred,
                                 model = "Random Forest (rxDForest)")

print("Scoring Random Forest (rxDForest) done")

In [ ]:
# Boosted Trees Scoring 
library("MicrosoftML")
## Make Predictions, then import them into R. 
boosted_prediction_sql <- RxSqlServerData(table = "Boosted_Prediction", stringsAsFactors = T,
                                          connectionString = connection_string)

rxPredict(modelObject = boosted_model,
          data = LoS_Test,
          outData = boosted_prediction_sql,
          extraVarsToWrite = c("lengthofstay", "eid"),
          overwrite = TRUE)

## Compute the performance metrics of the model.
boosted_prediction <- rxImport(boosted_prediction_sql)

boosted_metrics <- evaluate_model(observed = boosted_prediction$lengthofstay,
                                  predicted = boosted_prediction$Score,
                                  model = "Boosted Trees (rxFastTrees)")

print("Scoring Boosted Trees (rxFastTrees) done")